In [ ]:
del model
del tokenizer

import gc
gc.collect()

torch.cuda.empty_cache()
print("Video memory cleared")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

model_name = "deepseek-ai/deepseek-llm-7b-chat"
cache_dir = "E:/huggingface"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, cache_dir=cache_dir)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    cache_dir=cache_dir,
    use_safetensors=True
)

model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

def generate_response(user_input, max_new_tokens=3000):
    messages = [
        {"role": "user", "content": user_input}
    ]

    input_tensor = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        input_tensor,
        max_new_tokens=max_new_tokens,
        use_cache=True,
        synced_gpus=False,
        do_sample=False
    )

    response = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True, errors="replace")
    return response


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.02s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


# Zeroshot

In [13]:
messages = """You are a habit recognition expert.

Your task is to determine whether each sentence in a given list describes a habit.

The input is a list of 66 sentences, each enclosed in single quotation marks ('').  
Please evaluate the sentences in the exact order they appear in the list.

For each sentence, assign one of the following labels:  
- "habit" if the sentence describes a recurring or routine behavior,  
- "not a habit" if it does not.

The output should be a numbered list. Each item should include the sentence and your classification result in the following format:

[Number]. [Sentence] — [Label]

Here is a structured one‑shot example to show how to answer:

Example:
'He checks his phone before going to bed every night.' — habit  

Now, please process the list:


['Morgens nach dem Aufstehen trinke ich direkt einen Kafffe.', 'Immer wenn ich müde bin, mache ich Atemübungen. ', 'Wenn ich ein negatives Gefühl stark fühle, versuche ich es auszufühlen. ', 'Abends nach dem Abendbrot trinke ich ein Glas Wein. ', 'Wenn ich am Wochenende zum Bäcker gehe, mache ich einen Abstecher in mein Lieblingscafe und trinke einen Cappucino. ', 'Nach dem Aufstehen mache ich Kraftsport', 'Sobald ich eine Aufgabe sehe, trage ich sie in Notion ein. ', 'Wenn mir in Teams angezeigt wird, dass sich ein Kollege im EvalGiGe Meeting befindet, dann trete ich dem Meeting bei. ', 'Wenn mich nach Feierabend noch niemand daheim erwartet, dann spaziere ich nach Hause. ', 'Einmal im Monat verbringe ich einen Tag mit mir allein in Dresden. ', 'Morgens nach dem Aufstehen wasche ich mein Gesicht.', 'Morgens/Abends nach dem Gesichtwaschen putze ich meine Zähne.', 'Morgens/Abends nach dem Zähneputzen creme ich mein Gesicht ein.', 'Morgens nach dem Gesicht-Eincremen füttere ich die Katze.', 'Morgens nach der Katzenfütterung bekommt der Hund eine Kaustange.', 'Morgens nach der Tierfütterung trinke ich einen Kaffee.', 'Nach dem Joggen trinke ich einen Proteinshake.', 'Wenn ich Lust auf ein Bier/Wein habe trinke ich ein Malzbier', 'Bevor ich schlafe mache ich meine Ohrstöpsel ins Ohr.', 'Bevor ich schlafen gehe nehme ich meine Nahrungsergänzungsmittel', 'Wenn ich gestresst bin, gehe ich zum Kühlschrank und nehme mir etwas zu essen.', 'Nach dem Aufstehen mache ich mir ein Zitronenwasser.', 'Wenn ich mich abends ins Bett lege, scrolle ich sinnlos durch social media.', 'wenn ich ins DM gehe, kaufe ich mir was Süßes.', 'nach dem Aufstehen trinken und Medikamenteinnahme mach ich ein kurzes HIT Workout', 'nach dem Abendbrot wasche ich ab', 'Nach dem Aufstehen mache ich ein paar Sprachübungen', 'Ich prüfe ob ich meinen Schlüssel dabeihabe, wenn ich aus der  WOhnung gehe', 'Ich nehme VItamin D tabletten ein', 'Samstags putze ich das Bad.', 'Wenn es nicht regnet, gieße ich im Sommer jeden Tag den Apfelbaum.', 'Nach dem Abendbrot an einem stressigen Tag esse ich gern ein Stück Schokolade', 'Nach dem Sport am Abend gehe ich oft erst spät ins Bett ', 'Ich habe mir heute eine neue Serie auf Netflix angesehen.', 'Ich war gestern im Kino', 'Ich habe vor, einen neuen Getränkegeschmack auszuprobieren', 'Ich plane, morgen ins Schwimmbad zu gehen.', 'Ich habe am Wochenende meine Freunde in einer anderen Stadt besucht.', 'Ich habe heute zum ersten Mal einen Podcast gehört.', 'Ich habe heute zum ersten Mal ein Selfie mit meiner neuen Kamera gemacht.', 'Heute habe ich einen langen Spaziergang durch eine fremde Gegend gemacht.', 'Ich habe letzte Woche meine Wohnung umdekoriert.', 'Letzte Woche war ich auf der Hochzeit meiner besten Freundin', 'Letzte Woche war ich auf der Beerdigung eines guten Freundes.', 'Ich habe vor, nächsten Mittwoch zum Arzt zu gehen', 'Mir geht es jetzt sehr schlecht, weil mein Magen weh tut', 'Ich bin an die Landschaft in den Bergen gewöhnt', 'Letzten Mittwoch hat es plötzlich geregnet, also bin ich in ein Café gegangen, das ich vorher nicht kannte, um Schutz vor dem Regen zu suchen, und ich habe festgestellt, dass es mir wirklich gut gefallen hat.', 'Heute Morgen fühlte ich mich plötzlich sehr müde', 'Haferbrei schmeckt besser, wenn man Blaubeeren dazu gibt', 'Es regnet heute, deshalb möchte ich nicht ins Fitnessstudio gehen', 'Ich habe mir den Rap einmal angehört und er klang ziemlich gut.', 'Heute habe ich versucht zu lernen, wie man einen Kuchen backt', 'Ich habe heute viele neue Freunde kennengelernt', 'Ich habe heute viele neue Möbel zur Einrichtung meines neuen Hauses bestellt.', 'Wenn ich versuche zu bezahlen, friert die Webseite aus unbekannten Gründen immer wieder ein', 'Vielleicht plane ich, in Zukunft zum Militär zu gehen.', 'Ich habe gestern einen Film auf Englisch ohne Untertitel gesehen.', 'Ich habe mir heute eine neue Pflanze gekauft.', 'Ich jogge', 'Letzte Woche war ich joggen.', 'Ich habe einmal Tischtennis gespielt und es nie wieder angefasst', 'Ich liebe Pommes Frites', 'Ich habe gestern Pommes gegessen', 'Ich mag Eis, aber nicht jedes Mal.', 'Gestern habe ich zum ersten Mal Yoga gemacht.']"""


response = generate_response(messages)
print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[1] 'Morgens nach dem Aufstehen trinke ich direkt einen Kafffe.' — habit
[2] 'Immer wenn ich m�de bin, mache ich Atem�bungen.' — habit
[3] 'Wenn ich ein negatives Gef�hl stark f�hle, versuche ich es auszuf�hlen.' — not a habit
[4] 'Abends nach dem Abendbrot trinke ich ein Glas Wein.' — habit
[5] 'Wenn ich am Wochenende zum Bäcker gehe, mache ich einen Abstecher in mein Lieblingscafe und trinke einen Cappucino.' — habit
[6] 'Nach dem Aufstehen mache ich Kraftsport.' — habit
[7] 'Sobald ich eine Aufgabe sehe, trage ich sie in Notion ein.' — habit
[8] 'Wenn mir in Teams angezeigt wird, dass sich ein Kollege im EvalGiGe Meeting befindet, dann trete ich dem Meeting bei.' — habit
[9] 'Wenn mich nach Feierabend noch niemand daheim erwartet, dann spaziere ich nach Hause.' — habit
[10] 'Einmal im Monat verbringe ich einen Tag mit mir allein in Dresden.' — not a habit
[11] 'Morgens nach dem Aufstehen wasche ich mein Gesicht.' — habit
[12] 'Morgens/Abends nach dem Gesichtwaschen putze ich meine Z

# Oneshot

In [ ]:
messages = """You are a habit recognition expert.

Your task is to determine whether each sentence in a given list describes a habit.

The input is a list of 66 sentences, each enclosed in single quotation marks ('').  
Please evaluate the sentences in the exact order they appear in the list.

For each sentence, assign one of the following labels:  
- "habit" if the sentence describes a recurring or routine behavior,  
- "not a habit" if it does not.

The output should be a numbered list. Each item should include the sentence and your classification result in the following format:

[Number]. [Sentence] — [Label]

Here is a structured one‑shot example to show how to answer:

Example:
'He checks his phone before going to bed every night.' — habit  

Now, please process the list:


['Morgens nach dem Aufstehen trinke ich direkt einen Kafffe.', 'Immer wenn ich müde bin, mache ich Atemübungen. ', 'Wenn ich ein negatives Gefühl stark fühle, versuche ich es auszufühlen. ', 'Abends nach dem Abendbrot trinke ich ein Glas Wein. ', 'Wenn ich am Wochenende zum Bäcker gehe, mache ich einen Abstecher in mein Lieblingscafe und trinke einen Cappucino. ', 'Nach dem Aufstehen mache ich Kraftsport', 'Sobald ich eine Aufgabe sehe, trage ich sie in Notion ein. ', 'Wenn mir in Teams angezeigt wird, dass sich ein Kollege im EvalGiGe Meeting befindet, dann trete ich dem Meeting bei. ', 'Wenn mich nach Feierabend noch niemand daheim erwartet, dann spaziere ich nach Hause. ', 'Einmal im Monat verbringe ich einen Tag mit mir allein in Dresden. ', 'Morgens nach dem Aufstehen wasche ich mein Gesicht.', 'Morgens/Abends nach dem Gesichtwaschen putze ich meine Zähne.', 'Morgens/Abends nach dem Zähneputzen creme ich mein Gesicht ein.', 'Morgens nach dem Gesicht-Eincremen füttere ich die Katze.', 'Morgens nach der Katzenfütterung bekommt der Hund eine Kaustange.', 'Morgens nach der Tierfütterung trinke ich einen Kaffee.', 'Nach dem Joggen trinke ich einen Proteinshake.', 'Wenn ich Lust auf ein Bier/Wein habe trinke ich ein Malzbier', 'Bevor ich schlafe mache ich meine Ohrstöpsel ins Ohr.', 'Bevor ich schlafen gehe nehme ich meine Nahrungsergänzungsmittel', 'Wenn ich gestresst bin, gehe ich zum Kühlschrank und nehme mir etwas zu essen.', 'Nach dem Aufstehen mache ich mir ein Zitronenwasser.', 'Wenn ich mich abends ins Bett lege, scrolle ich sinnlos durch social media.', 'wenn ich ins DM gehe, kaufe ich mir was Süßes.', 'nach dem Aufstehen trinken und Medikamenteinnahme mach ich ein kurzes HIT Workout', 'nach dem Abendbrot wasche ich ab', 'Nach dem Aufstehen mache ich ein paar Sprachübungen', 'Ich prüfe ob ich meinen Schlüssel dabeihabe, wenn ich aus der  WOhnung gehe', 'Ich nehme VItamin D tabletten ein', 'Samstags putze ich das Bad.', 'Wenn es nicht regnet, gieße ich im Sommer jeden Tag den Apfelbaum.', 'Nach dem Abendbrot an einem stressigen Tag esse ich gern ein Stück Schokolade', 'Nach dem Sport am Abend gehe ich oft erst spät ins Bett ', 'Ich habe mir heute eine neue Serie auf Netflix angesehen.', 'Ich war gestern im Kino', 'Ich habe vor, einen neuen Getränkegeschmack auszuprobieren', 'Ich plane, morgen ins Schwimmbad zu gehen.', 'Ich habe am Wochenende meine Freunde in einer anderen Stadt besucht.', 'Ich habe heute zum ersten Mal einen Podcast gehört.', 'Ich habe heute zum ersten Mal ein Selfie mit meiner neuen Kamera gemacht.', 'Heute habe ich einen langen Spaziergang durch eine fremde Gegend gemacht.', 'Ich habe letzte Woche meine Wohnung umdekoriert.', 'Letzte Woche war ich auf der Hochzeit meiner besten Freundin', 'Letzte Woche war ich auf der Beerdigung eines guten Freundes.', 'Ich habe vor, nächsten Mittwoch zum Arzt zu gehen', 'Mir geht es jetzt sehr schlecht, weil mein Magen weh tut', 'Ich bin an die Landschaft in den Bergen gewöhnt', 'Letzten Mittwoch hat es plötzlich geregnet, also bin ich in ein Café gegangen, das ich vorher nicht kannte, um Schutz vor dem Regen zu suchen, und ich habe festgestellt, dass es mir wirklich gut gefallen hat.', 'Heute Morgen fühlte ich mich plötzlich sehr müde', 'Haferbrei schmeckt besser, wenn man Blaubeeren dazu gibt', 'Es regnet heute, deshalb möchte ich nicht ins Fitnessstudio gehen', 'Ich habe mir den Rap einmal angehört und er klang ziemlich gut.', 'Heute habe ich versucht zu lernen, wie man einen Kuchen backt', 'Ich habe heute viele neue Freunde kennengelernt', 'Ich habe heute viele neue Möbel zur Einrichtung meines neuen Hauses bestellt.', 'Wenn ich versuche zu bezahlen, friert die Webseite aus unbekannten Gründen immer wieder ein', 'Vielleicht plane ich, in Zukunft zum Militär zu gehen.', 'Ich habe gestern einen Film auf Englisch ohne Untertitel gesehen.', 'Ich habe mir heute eine neue Pflanze gekauft.', 'Ich jogge', 'Letzte Woche war ich joggen.', 'Ich habe einmal Tischtennis gespielt und es nie wieder angefasst', 'Ich liebe Pommes Frites', 'Ich habe gestern Pommes gegessen', 'Ich mag Eis, aber nicht jedes Mal.', 'Gestern habe ich zum ersten Mal Yoga gemacht.']"""


response = generate_response(messages)
print(response)

# Fewshot

In [ ]:
messages = """You are a habit recognition expert.

Your task is to determine whether each sentence in a given list describes a habit.

The input is a list of 66 sentences, each enclosed in single quotation marks ('').  
Please evaluate the sentences in the exact order they appear in the list.

For each sentence, assign one of the following labels:  
- "habit" if the sentence describes a recurring or routine behavior,  
- "not a habit" if it does not.

The output should be a numbered list. Each item should include the sentence and your classification result in the following format:

[Number]. [Sentence] — [Label]
Here are our examples (two habits, two non‑habits):

Example 1:
'He checks his phone before going to bed every night.' — habit  

Example 2:
'I always read a book before going to sleep.' — habit  

Example 3:
'She visited Paris last summer.' — not a habit  

Example  4:
'I jog.' — not a habit  

Now, please process the list:


['Morgens nach dem Aufstehen trinke ich direkt einen Kafffe.', 'Immer wenn ich müde bin, mache ich Atemübungen. ', 'Wenn ich ein negatives Gefühl stark fühle, versuche ich es auszufühlen. ', 'Abends nach dem Abendbrot trinke ich ein Glas Wein. ', 'Wenn ich am Wochenende zum Bäcker gehe, mache ich einen Abstecher in mein Lieblingscafe und trinke einen Cappucino. ', 'Nach dem Aufstehen mache ich Kraftsport', 'Sobald ich eine Aufgabe sehe, trage ich sie in Notion ein. ', 'Wenn mir in Teams angezeigt wird, dass sich ein Kollege im EvalGiGe Meeting befindet, dann trete ich dem Meeting bei. ', 'Wenn mich nach Feierabend noch niemand daheim erwartet, dann spaziere ich nach Hause. ', 'Einmal im Monat verbringe ich einen Tag mit mir allein in Dresden. ', 'Morgens nach dem Aufstehen wasche ich mein Gesicht.', 'Morgens/Abends nach dem Gesichtwaschen putze ich meine Zähne.', 'Morgens/Abends nach dem Zähneputzen creme ich mein Gesicht ein.', 'Morgens nach dem Gesicht-Eincremen füttere ich die Katze.', 'Morgens nach der Katzenfütterung bekommt der Hund eine Kaustange.', 'Morgens nach der Tierfütterung trinke ich einen Kaffee.', 'Nach dem Joggen trinke ich einen Proteinshake.', 'Wenn ich Lust auf ein Bier/Wein habe trinke ich ein Malzbier', 'Bevor ich schlafe mache ich meine Ohrstöpsel ins Ohr.', 'Bevor ich schlafen gehe nehme ich meine Nahrungsergänzungsmittel', 'Wenn ich gestresst bin, gehe ich zum Kühlschrank und nehme mir etwas zu essen.', 'Nach dem Aufstehen mache ich mir ein Zitronenwasser.', 'Wenn ich mich abends ins Bett lege, scrolle ich sinnlos durch social media.', 'wenn ich ins DM gehe, kaufe ich mir was Süßes.', 'nach dem Aufstehen trinken und Medikamenteinnahme mach ich ein kurzes HIT Workout', 'nach dem Abendbrot wasche ich ab', 'Nach dem Aufstehen mache ich ein paar Sprachübungen', 'Ich prüfe ob ich meinen Schlüssel dabeihabe, wenn ich aus der  WOhnung gehe', 'Ich nehme VItamin D tabletten ein', 'Samstags putze ich das Bad.', 'Wenn es nicht regnet, gieße ich im Sommer jeden Tag den Apfelbaum.', 'Nach dem Abendbrot an einem stressigen Tag esse ich gern ein Stück Schokolade', 'Nach dem Sport am Abend gehe ich oft erst spät ins Bett ', 'Ich habe mir heute eine neue Serie auf Netflix angesehen.', 'Ich war gestern im Kino', 'Ich habe vor, einen neuen Getränkegeschmack auszuprobieren', 'Ich plane, morgen ins Schwimmbad zu gehen.', 'Ich habe am Wochenende meine Freunde in einer anderen Stadt besucht.', 'Ich habe heute zum ersten Mal einen Podcast gehört.', 'Ich habe heute zum ersten Mal ein Selfie mit meiner neuen Kamera gemacht.', 'Heute habe ich einen langen Spaziergang durch eine fremde Gegend gemacht.', 'Ich habe letzte Woche meine Wohnung umdekoriert.', 'Letzte Woche war ich auf der Hochzeit meiner besten Freundin', 'Letzte Woche war ich auf der Beerdigung eines guten Freundes.', 'Ich habe vor, nächsten Mittwoch zum Arzt zu gehen', 'Mir geht es jetzt sehr schlecht, weil mein Magen weh tut', 'Ich bin an die Landschaft in den Bergen gewöhnt', 'Letzten Mittwoch hat es plötzlich geregnet, also bin ich in ein Café gegangen, das ich vorher nicht kannte, um Schutz vor dem Regen zu suchen, und ich habe festgestellt, dass es mir wirklich gut gefallen hat.', 'Heute Morgen fühlte ich mich plötzlich sehr müde', 'Haferbrei schmeckt besser, wenn man Blaubeeren dazu gibt', 'Es regnet heute, deshalb möchte ich nicht ins Fitnessstudio gehen', 'Ich habe mir den Rap einmal angehört und er klang ziemlich gut.', 'Heute habe ich versucht zu lernen, wie man einen Kuchen backt', 'Ich habe heute viele neue Freunde kennengelernt', 'Ich habe heute viele neue Möbel zur Einrichtung meines neuen Hauses bestellt.', 'Wenn ich versuche zu bezahlen, friert die Webseite aus unbekannten Gründen immer wieder ein', 'Vielleicht plane ich, in Zukunft zum Militär zu gehen.', 'Ich habe gestern einen Film auf Englisch ohne Untertitel gesehen.', 'Ich habe mir heute eine neue Pflanze gekauft.', 'Ich jogge', 'Letzte Woche war ich joggen.', 'Ich habe einmal Tischtennis gespielt und es nie wieder angefasst', 'Ich liebe Pommes Frites', 'Ich habe gestern Pommes gegessen', 'Ich mag Eis, aber nicht jedes Mal.', 'Gestern habe ich zum ersten Mal Yoga gemacht.']"""


response = generate_response(messages)
print(response)

# Definition

In [ ]:
messages = """You are a habit recognition expert.

habit is a process by which a stimulus generates an impulse to act as a result of a learned stimulus-response association. Habit-generated impulses may compete or combine with impulses and inhibitions arising from other sources, including conscious decisionmaking, to influence responses, and need not generate behaviour.

Your task is to determine whether each sentence in a given list describes a habit.

The input is a list of 66 sentences, each enclosed in single quotation marks ('').  
Please evaluate the sentences in the exact order they appear in the list.

For each sentence, assign one of the following labels:  
- "habit" if the sentence describes a recurring or routine behavior,  
- "not a habit" if it does not.

The output should be a numbered list. Each item should include the sentence and your classification result in the following format:

[Number]. [Sentence] — [Label]

Now, please process the list:


['Morgens nach dem Aufstehen trinke ich direkt einen Kafffe.', 'Immer wenn ich müde bin, mache ich Atemübungen. ', 'Wenn ich ein negatives Gefühl stark fühle, versuche ich es auszufühlen. ', 'Abends nach dem Abendbrot trinke ich ein Glas Wein. ', 'Wenn ich am Wochenende zum Bäcker gehe, mache ich einen Abstecher in mein Lieblingscafe und trinke einen Cappucino. ', 'Nach dem Aufstehen mache ich Kraftsport', 'Sobald ich eine Aufgabe sehe, trage ich sie in Notion ein. ', 'Wenn mir in Teams angezeigt wird, dass sich ein Kollege im EvalGiGe Meeting befindet, dann trete ich dem Meeting bei. ', 'Wenn mich nach Feierabend noch niemand daheim erwartet, dann spaziere ich nach Hause. ', 'Einmal im Monat verbringe ich einen Tag mit mir allein in Dresden. ', 'Morgens nach dem Aufstehen wasche ich mein Gesicht.', 'Morgens/Abends nach dem Gesichtwaschen putze ich meine Zähne.', 'Morgens/Abends nach dem Zähneputzen creme ich mein Gesicht ein.', 'Morgens nach dem Gesicht-Eincremen füttere ich die Katze.', 'Morgens nach der Katzenfütterung bekommt der Hund eine Kaustange.', 'Morgens nach der Tierfütterung trinke ich einen Kaffee.', 'Nach dem Joggen trinke ich einen Proteinshake.', 'Wenn ich Lust auf ein Bier/Wein habe trinke ich ein Malzbier', 'Bevor ich schlafe mache ich meine Ohrstöpsel ins Ohr.', 'Bevor ich schlafen gehe nehme ich meine Nahrungsergänzungsmittel', 'Wenn ich gestresst bin, gehe ich zum Kühlschrank und nehme mir etwas zu essen.', 'Nach dem Aufstehen mache ich mir ein Zitronenwasser.', 'Wenn ich mich abends ins Bett lege, scrolle ich sinnlos durch social media.', 'wenn ich ins DM gehe, kaufe ich mir was Süßes.', 'nach dem Aufstehen trinken und Medikamenteinnahme mach ich ein kurzes HIT Workout', 'nach dem Abendbrot wasche ich ab', 'Nach dem Aufstehen mache ich ein paar Sprachübungen', 'Ich prüfe ob ich meinen Schlüssel dabeihabe, wenn ich aus der  WOhnung gehe', 'Ich nehme VItamin D tabletten ein', 'Samstags putze ich das Bad.', 'Wenn es nicht regnet, gieße ich im Sommer jeden Tag den Apfelbaum.', 'Nach dem Abendbrot an einem stressigen Tag esse ich gern ein Stück Schokolade', 'Nach dem Sport am Abend gehe ich oft erst spät ins Bett ', 'Ich habe mir heute eine neue Serie auf Netflix angesehen.', 'Ich war gestern im Kino', 'Ich habe vor, einen neuen Getränkegeschmack auszuprobieren', 'Ich plane, morgen ins Schwimmbad zu gehen.', 'Ich habe am Wochenende meine Freunde in einer anderen Stadt besucht.', 'Ich habe heute zum ersten Mal einen Podcast gehört.', 'Ich habe heute zum ersten Mal ein Selfie mit meiner neuen Kamera gemacht.', 'Heute habe ich einen langen Spaziergang durch eine fremde Gegend gemacht.', 'Ich habe letzte Woche meine Wohnung umdekoriert.', 'Letzte Woche war ich auf der Hochzeit meiner besten Freundin', 'Letzte Woche war ich auf der Beerdigung eines guten Freundes.', 'Ich habe vor, nächsten Mittwoch zum Arzt zu gehen', 'Mir geht es jetzt sehr schlecht, weil mein Magen weh tut', 'Ich bin an die Landschaft in den Bergen gewöhnt', 'Letzten Mittwoch hat es plötzlich geregnet, also bin ich in ein Café gegangen, das ich vorher nicht kannte, um Schutz vor dem Regen zu suchen, und ich habe festgestellt, dass es mir wirklich gut gefallen hat.', 'Heute Morgen fühlte ich mich plötzlich sehr müde', 'Haferbrei schmeckt besser, wenn man Blaubeeren dazu gibt', 'Es regnet heute, deshalb möchte ich nicht ins Fitnessstudio gehen', 'Ich habe mir den Rap einmal angehört und er klang ziemlich gut.', 'Heute habe ich versucht zu lernen, wie man einen Kuchen backt', 'Ich habe heute viele neue Freunde kennengelernt', 'Ich habe heute viele neue Möbel zur Einrichtung meines neuen Hauses bestellt.', 'Wenn ich versuche zu bezahlen, friert die Webseite aus unbekannten Gründen immer wieder ein', 'Vielleicht plane ich, in Zukunft zum Militär zu gehen.', 'Ich habe gestern einen Film auf Englisch ohne Untertitel gesehen.', 'Ich habe mir heute eine neue Pflanze gekauft.', 'Ich jogge', 'Letzte Woche war ich joggen.', 'Ich habe einmal Tischtennis gespielt und es nie wieder angefasst', 'Ich liebe Pommes Frites', 'Ich habe gestern Pommes gegessen', 'Ich mag Eis, aber nicht jedes Mal.', 'Gestern habe ich zum ersten Mal Yoga gemacht.']"""


response = generate_response(messages)
print(response)

# Definition and Oneshot

In [ ]:
messages = """You are a habit recognition expert.

habit is a process by which a stimulus generates an impulse to act as a result of a learned stimulus-response association. Habit-generated impulses may compete or combine with impulses and inhibitions arising from other sources, including conscious decisionmaking, to influence responses, and need not generate behaviour.

Your task is to determine whether each sentence in a given list describes a habit.

The input is a list of 66 sentences, each enclosed in single quotation marks ('').  
Please evaluate the sentences in the exact order they appear in the list.

For each sentence, assign one of the following labels:  
- "habit" if the sentence describes a recurring or routine behavior,  
- "not a habit" if it does not.

The output should be a numbered list. Each item should include the sentence and your classification result in the following format:

[Number]. [Sentence] — [Label]

Here is a structured one‑shot example to show how to answer:

Example:
'He checks his phone before going to bed every night.' — habit  

Now, please process the list:


['Morgens nach dem Aufstehen trinke ich direkt einen Kafffe.', 'Immer wenn ich müde bin, mache ich Atemübungen. ', 'Wenn ich ein negatives Gefühl stark fühle, versuche ich es auszufühlen. ', 'Abends nach dem Abendbrot trinke ich ein Glas Wein. ', 'Wenn ich am Wochenende zum Bäcker gehe, mache ich einen Abstecher in mein Lieblingscafe und trinke einen Cappucino. ', 'Nach dem Aufstehen mache ich Kraftsport', 'Sobald ich eine Aufgabe sehe, trage ich sie in Notion ein. ', 'Wenn mir in Teams angezeigt wird, dass sich ein Kollege im EvalGiGe Meeting befindet, dann trete ich dem Meeting bei. ', 'Wenn mich nach Feierabend noch niemand daheim erwartet, dann spaziere ich nach Hause. ', 'Einmal im Monat verbringe ich einen Tag mit mir allein in Dresden. ', 'Morgens nach dem Aufstehen wasche ich mein Gesicht.', 'Morgens/Abends nach dem Gesichtwaschen putze ich meine Zähne.', 'Morgens/Abends nach dem Zähneputzen creme ich mein Gesicht ein.', 'Morgens nach dem Gesicht-Eincremen füttere ich die Katze.', 'Morgens nach der Katzenfütterung bekommt der Hund eine Kaustange.', 'Morgens nach der Tierfütterung trinke ich einen Kaffee.', 'Nach dem Joggen trinke ich einen Proteinshake.', 'Wenn ich Lust auf ein Bier/Wein habe trinke ich ein Malzbier', 'Bevor ich schlafe mache ich meine Ohrstöpsel ins Ohr.', 'Bevor ich schlafen gehe nehme ich meine Nahrungsergänzungsmittel', 'Wenn ich gestresst bin, gehe ich zum Kühlschrank und nehme mir etwas zu essen.', 'Nach dem Aufstehen mache ich mir ein Zitronenwasser.', 'Wenn ich mich abends ins Bett lege, scrolle ich sinnlos durch social media.', 'wenn ich ins DM gehe, kaufe ich mir was Süßes.', 'nach dem Aufstehen trinken und Medikamenteinnahme mach ich ein kurzes HIT Workout', 'nach dem Abendbrot wasche ich ab', 'Nach dem Aufstehen mache ich ein paar Sprachübungen', 'Ich prüfe ob ich meinen Schlüssel dabeihabe, wenn ich aus der  WOhnung gehe', 'Ich nehme VItamin D tabletten ein', 'Samstags putze ich das Bad.', 'Wenn es nicht regnet, gieße ich im Sommer jeden Tag den Apfelbaum.', 'Nach dem Abendbrot an einem stressigen Tag esse ich gern ein Stück Schokolade', 'Nach dem Sport am Abend gehe ich oft erst spät ins Bett ', 'Ich habe mir heute eine neue Serie auf Netflix angesehen.', 'Ich war gestern im Kino', 'Ich habe vor, einen neuen Getränkegeschmack auszuprobieren', 'Ich plane, morgen ins Schwimmbad zu gehen.', 'Ich habe am Wochenende meine Freunde in einer anderen Stadt besucht.', 'Ich habe heute zum ersten Mal einen Podcast gehört.', 'Ich habe heute zum ersten Mal ein Selfie mit meiner neuen Kamera gemacht.', 'Heute habe ich einen langen Spaziergang durch eine fremde Gegend gemacht.', 'Ich habe letzte Woche meine Wohnung umdekoriert.', 'Letzte Woche war ich auf der Hochzeit meiner besten Freundin', 'Letzte Woche war ich auf der Beerdigung eines guten Freundes.', 'Ich habe vor, nächsten Mittwoch zum Arzt zu gehen', 'Mir geht es jetzt sehr schlecht, weil mein Magen weh tut', 'Ich bin an die Landschaft in den Bergen gewöhnt', 'Letzten Mittwoch hat es plötzlich geregnet, also bin ich in ein Café gegangen, das ich vorher nicht kannte, um Schutz vor dem Regen zu suchen, und ich habe festgestellt, dass es mir wirklich gut gefallen hat.', 'Heute Morgen fühlte ich mich plötzlich sehr müde', 'Haferbrei schmeckt besser, wenn man Blaubeeren dazu gibt', 'Es regnet heute, deshalb möchte ich nicht ins Fitnessstudio gehen', 'Ich habe mir den Rap einmal angehört und er klang ziemlich gut.', 'Heute habe ich versucht zu lernen, wie man einen Kuchen backt', 'Ich habe heute viele neue Freunde kennengelernt', 'Ich habe heute viele neue Möbel zur Einrichtung meines neuen Hauses bestellt.', 'Wenn ich versuche zu bezahlen, friert die Webseite aus unbekannten Gründen immer wieder ein', 'Vielleicht plane ich, in Zukunft zum Militär zu gehen.', 'Ich habe gestern einen Film auf Englisch ohne Untertitel gesehen.', 'Ich habe mir heute eine neue Pflanze gekauft.', 'Ich jogge', 'Letzte Woche war ich joggen.', 'Ich habe einmal Tischtennis gespielt und es nie wieder angefasst', 'Ich liebe Pommes Frites', 'Ich habe gestern Pommes gegessen', 'Ich mag Eis, aber nicht jedes Mal.', 'Gestern habe ich zum ersten Mal Yoga gemacht.']"""


response = generate_response(messages)
print(response)

# Definition and Fewshot

In [ ]:
messages = """You are a habit recognition expert.

habit is a process by which a stimulus generates an impulse to act as a result of a learned stimulus-response association. Habit-generated impulses may compete or combine with impulses and inhibitions arising from other sources, including conscious decisionmaking, to influence responses, and need not generate behaviour.

Your task is to determine whether each sentence in a given list describes a habit.

The input is a list of 66 sentences, each enclosed in single quotation marks ('').  
Please evaluate the sentences in the exact order they appear in the list.

For each sentence, assign one of the following labels:  
- "habit" if the sentence describes a recurring or routine behavior,  
- "not a habit" if it does not.

The output should be a numbered list. Each item should include the sentence and your classification result in the following format:

[Number]. [Sentence] — [Label]

Here are our examples (two habits, two non‑habits):

Example 1:
'He checks his phone before going to bed every night.' — habit  

Example 2:
'I always read a book before going to sleep.' — habit  

Example 3:
'She visited Paris last summer.' — not a habit  

Example  4:
'I jog.' — not a habit    

Now, please process the list:


['Morgens nach dem Aufstehen trinke ich direkt einen Kafffe.', 'Immer wenn ich müde bin, mache ich Atemübungen. ', 'Wenn ich ein negatives Gefühl stark fühle, versuche ich es auszufühlen. ', 'Abends nach dem Abendbrot trinke ich ein Glas Wein. ', 'Wenn ich am Wochenende zum Bäcker gehe, mache ich einen Abstecher in mein Lieblingscafe und trinke einen Cappucino. ', 'Nach dem Aufstehen mache ich Kraftsport', 'Sobald ich eine Aufgabe sehe, trage ich sie in Notion ein. ', 'Wenn mir in Teams angezeigt wird, dass sich ein Kollege im EvalGiGe Meeting befindet, dann trete ich dem Meeting bei. ', 'Wenn mich nach Feierabend noch niemand daheim erwartet, dann spaziere ich nach Hause. ', 'Einmal im Monat verbringe ich einen Tag mit mir allein in Dresden. ', 'Morgens nach dem Aufstehen wasche ich mein Gesicht.', 'Morgens/Abends nach dem Gesichtwaschen putze ich meine Zähne.', 'Morgens/Abends nach dem Zähneputzen creme ich mein Gesicht ein.', 'Morgens nach dem Gesicht-Eincremen füttere ich die Katze.', 'Morgens nach der Katzenfütterung bekommt der Hund eine Kaustange.', 'Morgens nach der Tierfütterung trinke ich einen Kaffee.', 'Nach dem Joggen trinke ich einen Proteinshake.', 'Wenn ich Lust auf ein Bier/Wein habe trinke ich ein Malzbier', 'Bevor ich schlafe mache ich meine Ohrstöpsel ins Ohr.', 'Bevor ich schlafen gehe nehme ich meine Nahrungsergänzungsmittel', 'Wenn ich gestresst bin, gehe ich zum Kühlschrank und nehme mir etwas zu essen.', 'Nach dem Aufstehen mache ich mir ein Zitronenwasser.', 'Wenn ich mich abends ins Bett lege, scrolle ich sinnlos durch social media.', 'wenn ich ins DM gehe, kaufe ich mir was Süßes.', 'nach dem Aufstehen trinken und Medikamenteinnahme mach ich ein kurzes HIT Workout', 'nach dem Abendbrot wasche ich ab', 'Nach dem Aufstehen mache ich ein paar Sprachübungen', 'Ich prüfe ob ich meinen Schlüssel dabeihabe, wenn ich aus der  WOhnung gehe', 'Ich nehme VItamin D tabletten ein', 'Samstags putze ich das Bad.', 'Wenn es nicht regnet, gieße ich im Sommer jeden Tag den Apfelbaum.', 'Nach dem Abendbrot an einem stressigen Tag esse ich gern ein Stück Schokolade', 'Nach dem Sport am Abend gehe ich oft erst spät ins Bett ', 'Ich habe mir heute eine neue Serie auf Netflix angesehen.', 'Ich war gestern im Kino', 'Ich habe vor, einen neuen Getränkegeschmack auszuprobieren', 'Ich plane, morgen ins Schwimmbad zu gehen.', 'Ich habe am Wochenende meine Freunde in einer anderen Stadt besucht.', 'Ich habe heute zum ersten Mal einen Podcast gehört.', 'Ich habe heute zum ersten Mal ein Selfie mit meiner neuen Kamera gemacht.', 'Heute habe ich einen langen Spaziergang durch eine fremde Gegend gemacht.', 'Ich habe letzte Woche meine Wohnung umdekoriert.', 'Letzte Woche war ich auf der Hochzeit meiner besten Freundin', 'Letzte Woche war ich auf der Beerdigung eines guten Freundes.', 'Ich habe vor, nächsten Mittwoch zum Arzt zu gehen', 'Mir geht es jetzt sehr schlecht, weil mein Magen weh tut', 'Ich bin an die Landschaft in den Bergen gewöhnt', 'Letzten Mittwoch hat es plötzlich geregnet, also bin ich in ein Café gegangen, das ich vorher nicht kannte, um Schutz vor dem Regen zu suchen, und ich habe festgestellt, dass es mir wirklich gut gefallen hat.', 'Heute Morgen fühlte ich mich plötzlich sehr müde', 'Haferbrei schmeckt besser, wenn man Blaubeeren dazu gibt', 'Es regnet heute, deshalb möchte ich nicht ins Fitnessstudio gehen', 'Ich habe mir den Rap einmal angehört und er klang ziemlich gut.', 'Heute habe ich versucht zu lernen, wie man einen Kuchen backt', 'Ich habe heute viele neue Freunde kennengelernt', 'Ich habe heute viele neue Möbel zur Einrichtung meines neuen Hauses bestellt.', 'Wenn ich versuche zu bezahlen, friert die Webseite aus unbekannten Gründen immer wieder ein', 'Vielleicht plane ich, in Zukunft zum Militär zu gehen.', 'Ich habe gestern einen Film auf Englisch ohne Untertitel gesehen.', 'Ich habe mir heute eine neue Pflanze gekauft.', 'Ich jogge', 'Letzte Woche war ich joggen.', 'Ich habe einmal Tischtennis gespielt und es nie wieder angefasst', 'Ich liebe Pommes Frites', 'Ich habe gestern Pommes gegessen', 'Ich mag Eis, aber nicht jedes Mal.', 'Gestern habe ich zum ersten Mal Yoga gemacht.']"""


response = generate_response(messages)
print(response)